In [7]:
# %pip install pinecone-client
# %pip install python-dotenv
# %pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 6.8 MB/s eta 0:00:000:00:01
Note: you may need to restart the kernel to use updated packages.


In [14]:
from openai import OpenAI, AzureOpenAI
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

True

In [12]:
from pinecone import Pinecone

# pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
# index = pc.Index("test-pdf")

/Users/ogb/projects/pdfai/backend/venv/lib/python3.9/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [8]:
# from openai import OpenAI
# client = OpenAI()

# response = client.embeddings.create(
#     input="Your text string goes here",
#     model="text-embedding-3-large"
# )

# print(response.data[0].embedding)

[0.0038017325568944216, -0.010467467829585075, -0.003118067281320691, 0.045824043452739716, 0.01688837632536888, -0.010218022391200066, -0.033647410571575165, 0.04719137027859688, -0.013303753919899464, 0.015003678388893604, 0.018274184316396713, 0.024870628491044044, -0.02950846590101719, -0.009271053597331047, -0.01054137758910656, 5.4530089983018115e-05, -0.021396871656179428, -0.004078893922269344, -0.02819656766951084, -0.024926060810685158, 0.03139316290616989, 0.020546909421682358, -0.0726163238286972, 0.03813742846250534, -0.004531591199338436, 0.016112323850393295, -0.036345116794109344, 0.012001094408333302, 0.011437533423304558, -0.012102720327675343, 0.03597556799650192, 0.022043582051992416, 0.033462636172771454, 0.004459991119801998, -0.002283118199557066, -0.01551180798560381, 0.018846984952688217, 0.038913480937480927, 0.0026284153573215008, 0.02010345086455345, 0.004092752002179623, 0.006674973759800196, -0.04774569347500801, 0.002265795599669218, 0.00916018895804882, 

In [15]:
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = "2024-02-01",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

response = client.embeddings.create(
    input="Your text string goes here",
    model="ada_gcal"
)

print(response.data[0].embedding)

[-0.006929283495992422, -0.005336422007530928, 0.01187589205801487, -0.025001877918839455, -0.02469271421432495, 0.039787933230400085, -0.010101565159857273, -0.00940258800983429, -0.013186474330723286, -0.009940262883901596, -0.011680984869599342, 0.007870214991271496, -0.014087079092860222, 0.007769400719553232, 0.010168774053454399, -0.005033980123698711, 0.02294527180492878, -0.001512210350483656, 0.014933916740119457, -0.01026286743581295, 0.004862596280872822, 0.012460612691938877, 0.004909642972052097, 0.010807262733578682, -0.006603318266570568, -0.00035746971843764186, 0.0055783758871257305, -0.012494217604398727, 0.0163721963763237, 0.004529910162091255, 0.0065629929304122925, -0.007070423103868961, -0.015216195955872536, -0.006589876487851143, -0.01872452348470688, 0.0040796073153615, 0.003212606767192483, -0.018993360921740532, 0.030163554474711418, -0.007594656199216843, 0.008233144879341125, 0.00942275021225214, -0.0010887914104387164, -0.00040955696022138, -0.00883130822

In [ ]:
def generate_embeddings(text, model="ada_gcal"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# df_bills['ada_v2'] = df_bills["text"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002'))

In [16]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("./pdf/grokking.pdf")
pages = loader.load_and_split()

In [18]:
print(len(pages))
print()

full_document = ""

for page in pages:
  full_document += page.page_content

print(full_document)

30

Towards Understanding Grokking:
An Effective Theory of Representation Learning
Ziming Liu, Ouail Kitouni, Niklas Nolte, Eric J. Michaud, Max Tegmark, Mike Williams
Department of Physics, Institute for AI and Fundamental Interactions, MIT
{zmliu,kitouni,nnolte,ericjm,tegmark,mwill}@mit.edu
Abstract
We aim to understand grokking , a phenomenon where models generalize long after
overﬁtting their training set. We present both a microscopic analysis anchored by an
effective theory and a macroscopic analysis of phase diagrams describing learning
performance across hyperparameters. We ﬁnd that generalization originates from
structured representations whose training dynamics and dependence on training set
size can be predicted by our effective theory in a toy setting. We observe empirically
the presence of four learning phases: comprehension ,grokking ,memorization , and
confusion . We ﬁnd representation learning to occur only in a “Goldilocks zone”
(including comprehension and grokking) b

In [37]:
import re

# Regular expression pattern for section titles
pattern = r"\n\d+ [A-Z][a-zA-Z\s:]+\n"
# TODO: Write a regular expression pattern to match Abstract

# Find all matches
matches = re.findall(pattern, full_document)

matches.insert(0, "Abstract")

# Print all section titles
for match in matches:
    print(match.strip())
    print()

1 Introduction

2 Problem Setting

3 Why Generalization Occurs: Representations and Dynamics

4 Delayed Generalization: A Phase Diagram

5 Related work

6 Conclusion



In [47]:
pattern = r"\n\d+ [A-Z][a-zA-Z\s:]+\n"

# Split the text into sections
sections = re.split(pattern, full_document)

# number of sections should match with number of section titles (aka matches of regex pattern)
# print(sections[0])
# print(sections[2])
# print(len(sections))

# number of sections should match with number of section titles (aka matches of regex pattern)
print(len(sections))

# Create a dictionary to store section titles and contents
section_contents = {}

# Use zip to iterate over matches and sections simultaneously
for match, section in zip(matches, sections):
    section_title = match.strip()
    content = section.strip()
    section_contents[section_title] = content

section_contents.keys()


7


dict_keys(['Abstract', '1 Introduction', '2 Problem Setting', '3 Why Generalization Occurs: Representations and Dynamics', '4 Delayed Generalization: A Phase Diagram', '5 Related work', '6 Conclusion'])

In [50]:
section_contents["6 Conclusion"] #TODO: process this to ONLY the abstract content

'We have shown how, in both toy models and general settings, that representation enables generalization\nwhen it reﬂects structure in the data. We developed an effective theory of representation learning\ndynamics (in a toy setting) which predicts the critical dependence of learning on the training data\nfraction. We then presented four learning phases (comprehension, grokking, memorization and\nconfusion) which depend on the decoder capacity and learning speed (given by, among other things,\nlearning rate and weight decay) in decoder-only architectures. While we have mostly focused on a\ntoy model, we ﬁnd preliminary evidence that our results generalize to the setting of [1].\nOur work can be viewed as a step towards a statistical physics of deep learning , connecting the\n“microphysics” of low-level network dynamics with the “thermodynamics” of high-level model\nbehavior. We view the application of theoretical tools from physics, such as effective theories [ 24], to\nbe a rich area f